In [4]:
import pandas as pd
!pip install psmatching
import psmatching.match as psm
import numpy as np
from scipy import stats

In [6]:
df = pd.read_csv('data (2).csv')

In [7]:
model = "CASE ~ gender + age + activedays_per_week"


In [8]:
k = "1"

In [9]:
path = 'data (2).csv'

In [10]:
m = psm.PSMatch(path, model, k)


In [11]:
m.prepare_data()


Calculating propensity scores ... DONE!
Preparing data ... DONE!


In [12]:
m.match(caliper=None, replace=True)


Matching [1] controls to each case ...  DONE!
Writing matched data to file ... DONE!



In [13]:
m.evaluate()

Evaluating matches ...
	gender: PASSED
	age: PASSED
	activedays_per_week: PASSED

All variables were successfully matched!


True

In [14]:
a = m.df

In [20]:
print('before match:\n',m.df['CASE'].value_counts(),'\n')

before match:
 CASE
0    10
1    10
Name: count, dtype: int64 



In [15]:
mdt_id = m.matches.set_index('CASE_ID')
mdt = m.matched_data
mdt_all = mdt.join(mdt_id)
mdt_merged = pd.merge(mdt_all, df, left_on='CONTROL_MATCH_1', right_on='OPTUM_LAB_ID', how='left',suffixes=('_all', '_control'))
mdt_merged.to_csv('matched_merged.csv', index=True)

In [16]:
print('after match:\n',mdt_merged['CASE_all'].value_counts(),'\n')


after match:
 CASE_all
1    10
0     5
Name: count, dtype: int64 



In [17]:
y_1 = mdt_merged[mdt_merged['CASE_all']==1]['cv_all']
y_0 = mdt_merged[mdt_merged['CASE_all']==1]['cv_control']
ATT = y_1.mean() - y_0.mean()

In [18]:
t_statistic, p_value = stats.ttest_rel(y_1, y_0)


In [19]:
print(f": {ATT}")
print("T-statistic:", t_statistic)
print("P-value:", p_value)
if p_value<0.05:
    print("P-value:", p_value)
    print("has significant effect  ！")
else:
    print("has no significant effect ！")

: 0.010900000000000014
T-statistic: 6.04210219838755
P-value: 0.00019233122730137023
P-value: 0.00019233122730137023
has significant effect  ！
